# Agents

> The core abstraction for different FL Agents/Clients.

In [ ]:
#| default_exp federated.agents

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

'0.19.1'

In [ ]:
#| export
from fastcore.utils import *
from fastcore.all import *
import os
import json
from collections import defaultdict,OrderedDict
from copy import deepcopy
from enum import Enum
import torch
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from peft import *
from community import community_louvain
from fedai.utils import *
from fedai.client_selector import *
from fedai.optimizers import *
from fedai.data.core import LLMDataCollator
from tqdm import tqdm
import numpy as np
from loguru import logger
from fedai.utils import *
from fedai.metrics import *
from transformers import AutoTokenizer
from omegaconf.dictconfig import DictConfig
import numpy as np
import math
np.random.seed(42)
torch.manual_seed(42)

<torch._C.Generator>

In [ ]:
#| export
class AgentRole(Enum):
    SERVER = 1
    CLIENT = 2
    MARL = 3

## Base Agent

An agent is an entity that has a state and exist in an environment. In the case of Federated learning (FL), the agent's state is defined as its own model, data, criterion, optimizer. FL Focuses on distributed model training across multiple clients (agents), each with its local data. Clients **collaborate** to improve a global or shared model while keeping their data private. Communication is often periodic (e.g., every few training rounds). On the other hand, Multi-agent RL systems (MARL) Involves multiple agents interacting with an environment to learn policies for specific tasks (e.g., navigation, resource allocation). Each agent has a state also, but the state represntation might differ slightly from that of an FL agent. The data is often not preloaded as in FL rather, it's collected from the environemnt.

In [ ]:
#| export
class Agent:
    def __init__(self,
                 id,
                 cfg,
                 state= None,
                 role= AgentRole.CLIENT):
        
        self.id = id # each agent has a unique id
        self.cfg = cfg # contains all the configurations needed for the agent/trainer.
        self.state = state # A dictionary containing the state of the agent
        self.role = role # either a client or a server

In [ ]:
#| export
@patch
def init_agent(self: Agent):
    # Initialize the state of the agent. In FL Agent, this means making any adjustments to the model/optimizer/state_dict/...etc
    raise NotImplementedError

In [ ]:
#| export
@patch
def communicate(self: Agent, msg):
    raise NotImplementedError

In [ ]:
#| export
@patch
def update_state(self: Agent):
    raise NotImplementedError

In [ ]:
#| export
@patch
def save_state(self: Agent):
    # save the state of the agent to a file on disk (id, model, optimizer, loss_fn).
    raise NotImplementedError

In [ ]:
#| export
@patch
def clear_model(self: Agent):
    self.model = None

## MARL Agent

In [ ]:
class MARLAgent(Agent):
    def _sense(self, state):
        # sense the environment
        self.state = state

    def _decide(self):
    # Compute the next action(s) based on the current state and observations.
        pass

    def _act(self):
        pass
    

## FL Agent (FedAVG)

In [ ]:
#| export
class FLAgent(Agent):
    # A Federated Learning Agent implementing `FedAVG`.
    def __init__(self,
                 id, # the id of the agent
                 cfg, # the configuration of the agent.
                 state= None, # the state of the agent (model, optimizer, loss_fn), etc.
                 role= AgentRole.CLIENT, # the role of the agent (client or server)
                 block= None # The data block (local data of the FL Agent).
                 ):  
                 
        super().__init__(id, cfg, state, role)

        if self.role == AgentRole.CLIENT:
            self.train_ds, self.test_ds = block[0], block[1]
            
            for key, value in self.state.items():
                setattr(self, key, value)

            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
            self.train_loader = prepare_dl(self.cfg, self.train_ds)  # noqa: F405
            self.test_loader = prepare_dl(self.cfg, self.test_ds) # noqa: F405

            self.training_metrics = Metrics(list(self.cfg.training_metrics))  # noqa: F405
            self.test_metrics = Metrics(list(self.cfg.test_metrics))  # noqa: F405

            self.data_key, self.label_key = 'x', 'y'

In [ ]:
#| export 
@patch
def server_init(self: FLAgent, client_fn, client_selector, client_cls, loss_fn, writer):
    self.client_fn = client_fn
    self.client_selector = client_selector
    self.client_cls = client_cls
    self.loss_fn = loss_fn
    self.writer = writer
    self.latest_round = {}

Since data blocks are already on the disk, and since RL agents don't have a preloaded data blocks, we don't include the data in the FL agent's state. Another ratioanle behind this decision is that, state should contain dynamic objects that change over the interaction of the agents and data blocks are static in the case of FL agents, unless you are doing FL-RL Agents. 

For every client abstraction, whether it a base or any other type of federated client, it will initalize the training locally with a set of steps. This might include things like extracting the peft model out of the base model (in the case of LLMs clients). Also, it will terminate the local training with some steps, like saving the model state dictionary and so on.

In [ ]:
#| export
@patch
def runFL(self: FLAgent):
    res =  []
    all_ids = self.client_selector.select()
    
    for t in range(1, self.cfg.n_rounds + 1):
        lst_active_ids = all_ids[t]
        len_clients_ds = []
        
        train_res, test_res = self.evaluate(t)
        train_df, test_df = self.writer.write(lst_active_ids, train_res, test_res, t) 
        res.append((train_df, test_df))
        
        for id in lst_active_ids:
            client = self.client_fn(self.client_cls, self.cfg, id, self.latest_round, t, self.loss_fn)
            len_clients_ds.append(len(client.train_ds))
            
            self.communicate(client) 
            client.fit()

            client.communicate(self) 
            self.latest_round[id] = t 

        self.aggregate(lst_active_ids, t, len_clients_ds)
        
        
        
        
    self.writer.save(res)
    self.writer.finish()

    return res

In [ ]:
#| export
@patch
def evaluate(self: FLAgent, t):
    lst_train_res = []
    lst_test_res = []
    for id in range(self.cfg.num_clients):
        client = self.client_fn(self.client_cls, self.cfg, id, self.latest_round, t, self.loss_fn)
        
        res_train = client.evaluate_local(loader= 'train')
        lst_train_res.append(res_train)

        res_test = client.evaluate_local(loader= 'test')
        lst_test_res.append(res_test)
    return lst_train_res, lst_test_res    


We will adjust the string reprsntation of the client abstraction to make it more meaningful.

In [ ]:
#| export
@patch
def __str__(self: FLAgent) -> str:
    return f'''{self.__class__.__name__}: {self.__class__.__name__}
    Index : {self.id}
    Model: {self.model.__class__.__name__}
    Criterion: {self.criterion.__class__.__name__}
    Optimizer: {self.optimizer.__class__.__name__}'''


In [ ]:
#| export
@patch
def clear_model(self: FLAgent):
    self.model = None if hasattr(self, 'model') else None

In [ ]:
#| export
@patch
def get_batch(self: FLAgent, batch):
    return {k: v.to(self.device) for k, v in batch.items()}

In [ ]:
#| export
@patch
def _forward(self: FLAgent, batch):
    X, y = batch['x'], batch['y']
    outputs = self.model(X)
    loss = self.criterion(outputs, y)
    return loss, outputs

In [ ]:
#| export
@patch
def _closure(self: FLAgent, batch: dict) -> tuple:
    metrics = {k: 0 for k in list(self.cfg.training_metrics)}  # Ensure metrics is always defined

    try:
        loss, logits = self._forward(batch)
        probs = torch.nn.functional.softmax(logits, dim=-1)
        y_pred = probs.argmax(dim=-1)
        y_true = batch[self.label_key]

        if hasattr(self, "training_metrics") and self.cfg.training_metrics:
            if hasattr(self, "tokenizer"):
                metrics = self.training_metrics.compute(y_pred=y_pred, y_true=y_true, tokenizer=self.tokenizer)
            else:
                metrics = self.training_metrics.compute(y_pred=y_pred, y_true=y_true)

    except Exception as e:
        return torch.tensor(0.0, dtype=torch.float32, device=self.device), metrics  # Return safe values

    return loss, metrics


In [ ]:
#| export
@patch
def _run_batch(self: FLAgent, batch: dict) -> tuple:
    self.model.zero_grad()
    loss, metrics = self._closure(batch)

    if loss.item() == 0.0:
        return loss, metrics
    
    loss.backward()
    
    if self.cfg.model.grad_norm_clip:
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg.model.grad_norm_clip)

    self.optimizer.step()

    return loss, metrics

In [ ]:
#| export
@patch
def _run_epoch(self: FLAgent):

    for i, batch in enumerate(self.train_loader):
        batch = self.get_batch(batch)
        self._run_batch(batch)

In [ ]:
#| export
@patch
def fit(self: FLAgent) -> dict:
    
    self.model = self.model.to(self.device)
    self.model.train()
    for _ in range(self.cfg.local_epochs):
        self._run_epoch()


In [ ]:
#| export
@patch
def evaluate_local(self: FLAgent, loader= 'train') -> dict:
    total_loss = 0
    lst_metrics = []

    self.model = self.model.to(self.device)
    self.model.eval()
    num_eval = 0
    data_loader = self.train_loader if loader == 'train' else self.test_loader
    
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            batch = self.get_batch(batch)
            loss, metrics = self._closure(batch)                 

            if not math.isnan(loss.item()):
                total_loss += loss.item()  
                num_eval += len(batch[self.data_key])  # Ensure num_eval is updated
                lst_metrics.append(metrics)           
    
    avg_loss = total_loss / num_eval if num_eval > 0 else 0.0

    if lst_metrics:
        total_metrics = {k: sum(m.get(k, 0) for m in lst_metrics) / len(lst_metrics) for k in self.cfg.test_metrics}
    else:
        total_metrics = {k: 0.0 for k in self.cfg.test_metrics}

    return {"loss": avg_loss, "metrics": total_metrics}


In [ ]:
#| export
@patch
def save_state(self: FLAgent, state_dict):  # noqa: F811
    # save the model to self.cfg.save_dir/comm_round/f"local_output_{id}"/state.pth
    
    state_path = os.path.join(self.cfg.save_dir, 
                              str(self.t),
                              f"local_output_{self.id}",
                              "state.pth")
    if not os.path.exists(os.path.dirname(state_path)):
        os.makedirs(os.path.dirname(state_path))

    state_dict['model'] = self.model.state_dict()
    state_dict['optimizer'] = self.optimizer.state_dict()

    torch.save(state_dict, state_path)

    if self.role == AgentRole.CLIENT:
        save_space(self)


> To do: implement the communication process in **Protobuf**.

### Communication

Communication refers to the process of downloading and uploading models from the server and to the client. Since we are safeguarding against memory issues, we use sequential client processing and disk checkpointing as our tools.

In [ ]:
#| export
@patch
def communicate(self: Agent, another_agent: Agent):  # noqa: F811
    if self.role == AgentRole.CLIENT:
        self.save_state(self.state)

### Aggregation


The aggegation for `fedavg` is defined as:

$$m_t \leftarrow \sum_{k \in S_t} n_k$$
$$W_{global}^{(t + 1)} \leftarrow \sum_{k \in S_t} \frac{n_k}{m_t} w_k^{(t + 1)}$$

where $S_t$ is the set of active clients at communication round $t$.

In [ ]:
sum([1, 2])

3

In [ ]:
#| export
@patch
def aggregate(self: FLAgent, lst_active_ids, comm_round, len_clients_ds):
        
    m_t = sum(len_clients_ds)
    with torch.no_grad():
        for i, id in enumerate(lst_active_ids):
            state_path = os.path.join(self.cfg.save_dir, 
                                    str(comm_round),
                                    f"local_output_{id}",
                                    "state.pth")
            
            state = torch.load(state_path, weights_only=False)
            client_state_dict = state['model']

            if i == 0:
                global_model = {
                    key: torch.zeros_like(value) 
                    for key, value in client_state_dict.items()
                }

            n_k = len_clients_ds[i]
            weight =  n_k / m_t 

        
            for key in client_state_dict.keys():
                global_model[key].add_(weight * client_state_dict[key])


        server_state = {
            'model': global_model,
        }

        server_state_path = os.path.join(self.cfg.save_dir, 
                                    str(comm_round),
                                    "global_model",
                                    "state.pth")
        os.makedirs(os.path.dirname(server_state_path), exist_ok=True)

        torch.save(server_state, server_state_path)

    

## Fedu Agent (MTL)

This client uses **Multi-Task-Learning** scheme to personalize the models in a non-iid setting.

In [ ]:
#| export
class Fedu(FLAgent):
    def __init__(self, 
                 id,
                 cfg,
                 state= None,
                 role= AgentRole.CLIENT,
                 block= None):
        
        super().__init__(id, cfg, state, role, block)

        b = np.random.uniform(0,1,size=(self.cfg.num_clients, self.cfg.num_clients))
        b_symm = (b + b.T)/2
        b_symm[b_symm < 0.25] = 0
        self.alk_connection = b_symm

This Algorithm learns a model per client, and add a new aggregation scheme defined as

$$ W_{k}^{(t)} = W_{k, R}^{(t)} - \lambda \eta_2 \sum_{\ell \in N_k} a_{k, \ell} (W_{k, R}^{(t)} - W_{\ell, R}^{(t)})$$

where $\lambda$ is a regularization parameter and $\eta_2$ is defined as $$\eta_2 = \eta_1 * R $$

such that $\eta_1$ is the local learning rate, and $R$ is the number of lcoal iterations.

In [ ]:
#| export
@patch
def aggregate(self: Fedu, lst_active_ids, comm_round, len_clients_ds):

    global_lr = float(self.cfg.lr) * float(self.cfg.local_epochs)
    reg_param = self.cfg.lambda_
    
    with torch.no_grad():
        for i, id in enumerate(lst_active_ids):
            state_path = os.path.join(self.cfg.save_dir, 
                                    str(comm_round),
                                    f"local_output_{id}",
                                    "state.pth")
            
            state = torch.load(state_path, weights_only= False)
            client_state_dict = state['model']

            client_diff = {
                key: torch.zeros_like(value) 
                for key, value in client_state_dict.items()
            }
            
            for j, other_id in enumerate(lst_active_ids):
                if i == j:
                    continue
                other_state_path = os.path.join(self.cfg.save_dir,
                                                str(comm_round),
                                                f"local_output_{other_id}",
                                                "state.pth")
                
                other_state = torch.load(other_state_path, weights_only= False)
                other_state_dict = other_state['model']

                weight = self.alk_connection[int(id)][int(other_id)]
                for key in client_state_dict.keys():
                    client_diff[key].add_(weight * (client_state_dict[key] - other_state_dict[key]))

            for key in client_state_dict:
                client_state_dict[key].sub_(global_lr * reg_param * client_diff[key])

            clinet_state = {
                'model': client_state_dict,
            }

            agg_client_state_path = os.path.join(self.cfg.save_dir,
                                                 str(comm_round),
                                                 f"aggregated_model_{id}",
                                                 "state.pth")
            
            if not os.path.exists(os.path.dirname(agg_client_state_path)):
                os.makedirs(os.path.dirname(agg_client_state_path))

            torch.save(clinet_state, agg_client_state_path)

## DMTL-G (MTL)
Distributed Multi-Task Learning over Graphs: A game-theoretic approach

In [ ]:
#| export
class DMTL(FLAgent):
    def __init__(self, 
                 id,
                 cfg,
                 state= None,
                 role= AgentRole.CLIENT,
                 block= None):
        
        super().__init__(id, cfg, state, role, block)

We build a weihted graph where the weights are defined as:

$\begin{align}
\operatorname{Sim} = \sum_{i \in S} \alpha \cdot \operatorname{Sim}_{\text {head }}(C_j)+(1-\alpha) \cdot \operatorname{Sim}_{\mathrm{repr}}(C_j)
\end{align}$


$\begin{align}
\operatorname{Sim}_{\text {head }}(C_j)=\frac{1}{|C_j|^2} \sum_{k, \ell \in C_j} \left\|\bm{w}_k-\bm{w}_\ell\right\|, \nonumber \\ \operatorname{Sim}_{\text {repr }}(C_j)=\frac{1}{|C_j|^2} \sum_{k, \ell \in C_j} \cos \left(\bm{h}_k^{\text { }}, \bm{h}_\ell^{\text { }}\right)
\end{align}$


In [ ]:
#| export
@patch
def head_similarity(self: DMTL, model1, model2):
    pass

In [ ]:
#| export
@patch
def repr_similarity(self: DMTL, model1, model2):
    pass

In [ ]:
# #| export
# @patch
# def build_graph(self: DMTL, cfg, model1, model2):
#     h_sim = self.head_similarity(model1, model2)
#     r_sim = self.repr_similarity(model1, model2)
#     return (self.cfg.alpha) * h_sim + (1-self.cfg.alpha) * r_sim
#     graph = np.random.rand(cfg.num_clients, cfg.num_clients)
#     graph = graph / graph.sum(axis=1)[:, None]
#     return graph

Now, we have a wighted graph, we nedd to form the coalitions (detecting the communities). We do so by

In [ ]:
#| export
def get_coalitions(self: DMTL):
    return community_louvain.best_partition(self.G)


The aggregation rule here is two-folds:
- Representations are aggregated as:
  $$ h_c = \sum_{k \in C_{j}} \frac{\zeta_k}{\sum_{k \in C_{j}} \zeta_k}h_k$$

  where $\zeta$ is the shapely value.

- Classification heads are aggregated as:
$$  w_k^{(t+1)} = w_{k, R}^{(t)} - \lambda \eta_2 \sum_{\ell \in C_{j}} a_{k, \ell} (w_{k,R}^{(t)} - w_{\ell, R}^{(t)})$$

In [ ]:
#| export
def get_shapley_vals(self: DMTL):
    pass

In [ ]:
#| export
@patch
def aggregate(self: DMTL, lst_active_ids, comm_round, len_clients_ds):

    global_lr = float(self.cfg.lr) * float(self.cfg.local_epochs)
    reg_param = self.cfg.lambda_
    self.G = self.build_graph(self.cfg)
    partitions = self.get_coalitions()
    # shapely_values = self.get_shapley_vals()

    for k, v in partitions.items():
        for i, id in enumerate(v):
            state_path = os.path.join(self.cfg.save_dir, 
                                    str(comm_round),
                                    f"local_output_{id}",
                                    "state.pth")
            state = torch.load(state_path, weights_only= False)
            client_head = state['model']['head']
            client_repr = state['model']['repr']

            if i == 0:
                client_avg = {
                    key: torch.zeros_like(value) 
                    for key, value in client_head.items()
                }

            for j, other_id in enumerate(v):
                if i == j:
                    continue
                other_state_path = os.path.join(self.cfg.save_dir,
                                                str(comm_round),
                                                f"local_output_{other_id}",
                                                "state.pth")
                other_state = torch.load(other_state_path, weights_only= False)
                other_head = other_state['model']['head']
                other_repr = other_state['model']['repr']



        client_diff = {
            key: torch.zeros_like(value) 
            for key, value in client_head.items()
        }
        
        for j, other_id in enumerate(lst_active_ids):
            if i == j:
                continue
            other_state_path = os.path.join(self.cfg.save_dir,
                                            str(comm_round),
                                            f"local_output_{other_id}",
                                            "state.pth")
            
            other_state = torch.load(other_state_path, weights_only= False)
            other_state_dict = other_state['model']

            weight = self.alk_connection[int(id)][int(other_id)] #FIXME
            for key in client_state_dict.keys():
                client_diff[key].data += weight * (client_state_dict[key].data.clone() - other_state_dict[key].data.clone())

    for key in client_state_dict:
        client_state_dict[key].data -=  global_lr * reg_param * client_diff[key].data

    clinet_state = {
        'model': client_state_dict,
    }

    agg_client_state_path = os.path.join(self.cfg.save_dir, 
                                  str(comm_round),
                                  f"aggregated_model_{id}",
                                  "state.pth")
    if not os.path.exists(os.path.dirname(agg_client_state_path)):
        os.makedirs(os.path.dirname(agg_client_state_path))

    torch.save(clinet_state, agg_client_state_path)

Now, we need to align the represntation of the coalition with the local represntation. This process is done as follows:

$$\min_{\phi_i} \left\|\bm{\phi}_i(x)- \bm{h}_c\right\|^2 ; \hspace{0.3cm} \forall x \in D_k$$

We prevent the coallpse of reprenstations as follows:

$$\bm{h}_c^{(i+1)} = \beta \bm{h}_c^{(i)} + (1-\beta) \bm{\phi}_i(x^{(i)})$$

In [ ]:
#| export
def repr_alignment(self: DMTL):
    pass

## PEFT Agent

In [ ]:
#| export
class PeftAgent(FLAgent):
    def __init__(self,
                 cfg,
                 block,
                 id,
                 state= None,
                 role= "client",
                 **adapter_settings):
        super().__init__(cfg, block, id, state, role)


In [ ]:
#| export
@patch
def peftify(self: PeftAgent):
    # extract only the adapter's parameters from the model and store them in a dictionary
    self.params_dict_old = deepcopy(
        OrderedDict((name, param.detach()) for name, param in self.model.named_parameters() if
                    "default" in name))
    
    self.params_dict_new = deepcopy(self.params_dict_old)
    
    self.model.state_dict = (
        lambda instance, *_, **__: get_peft_model_state_dict(  # noqa: F405
            instance, self.params_dict_new, "default"
        )
    ).__get__(self.model, type(self.model))

In [ ]:
#| export
@patch 
def init_agent(self: PeftAgent):  # noqa: F811
    self.peftify()


In [ ]:
#| export
@patch
def save_state_(self: PeftAgent, epoch, local_dataset_len_dict, previously_selected_clients_set):  # noqa: F811
    # save the new adapter weights to disk
    self.save_state(epoch)

    local_dataset_len_dict[self.id] = len(self.block)
    older_adapter_weight = get_peft_model_state_dict(self.model, self.params_dict_old, "default")  # noqa: F405
    set_peft_model_state_dict(self.model, older_adapter_weight, "default")  # noqa: F405
    previously_selected_clients_set = previously_selected_clients_set | set({self.id})
    last_client_id = self.id

    return self.model, local_dataset_len_dict, previously_selected_clients_set, last_client_id

## Fed-Sophia Agent

In [ ]:
#| export
class FedSophiaAgent(FLAgent):
    def __init__(self,
                 id, # the id of the agent
                 cfg, # the configuration of the agent.
                 state= None, # the state of the agent (model, optimizer, loss_fn), etc.
                 role= AgentRole.CLIENT, # the role of the agent (client or server)
                 block= None):
        super().__init__(id, cfg, state, role, block)


In [ ]:
#| export
@patch
def train(self: FedSophiaAgent):
    trainer = self.trainer(self) 
    client_history = trainer.fit() 
    return client_history

## PADG Agent

In [ ]:
#| export
class PadgAgent(FLAgent):
    def __init__(self,
                 id, # the id of the agent
                 cfg, # the configuration of the agent.
                 state= None, # the state of the agent (model, optimizer, loss_fn), etc.
                 role= AgentRole.CLIENT, # the role of the agent (client or server)
                 block= None):
        super().__init__(id, cfg, state, role, block)

        if role == AgentRole.SERVER:
            self.connections = torch.from_numpy(generate_graph(self.cfg.num_clients))  # noqa: F405


In [ ]:
#| export
@patch
def apply_constraints(self: PadgAgent, 
                      graph, # (np.ndarray): The input matrix.
                      symmetrize=True, # (bool): If True, makes the matrix symmetric.
                      normalize=True, # (bool): If True, normalizes the matrix symmetrically.
                      threshold= 0, # (float or None): If provided, sets values below this threshold to 0.
                      diag_fill= 0): # (float or None): If provided, fills the diagonal with this value.
    

    # Symmetrize the matrix
    if symmetrize:
        graph = (graph + graph.T) / 2

    # Apply threshold to ensure non-negativity
    if threshold is not None:
        graph = torch.where(graph > threshold, graph, 0)

    # Normalize the matrix symmetrically
    if normalize:
        row_sums = graph.sum(axis=1, keepdims=True)
        col_sums = graph.sum(axis=0, keepdims=True)
        norm_factor = torch.sqrt(row_sums @ col_sums)  # Symmetric normalization factor
        graph = torch.divide(graph, norm_factor, where=norm_factor != 0)

    # Fill the diagonal
    if diag_fill is not None:
        torch.fill_diagonal(graph, diag_fill)

    return graph


In [ ]:
from fedai.models import *  # noqa: F403
model = MLP(dim_in= 784, dim_hidden= 64, dim_out= 10)  # noqa: F405

In [ ]:
import torch
rand_in = torch.rand(1, 1, 28, 28)
rand_in2 = torch.rand(1, 1, 28, 28)
out1 = model(rand_in)
out2 = model(rand_in2)

In [ ]:
# compute the KL divergence between the two outputs
dist1 = F.softmax(out1, dim= -1)
dist2 = F.softmax(out2, dim= -1)
F.kl_div(dist1, dist2, reduction= 'batchmean')

tensor(-2.3843, grad_fn=<DivBackward0>)

In [ ]:
#| export
@patch
def compute_probs(self: PadgAgent,
                  batch_size=32, # batch_size (int): Batch size for evaluation.
                  return_log_probs=True): # return_log_probs (bool): If True, return log-probabilities; otherwise, return probabilities.
    
    # Computes probabilities or log-probabilities across the entire dataset for a given model.
    # Ensure model is in evaluation mode
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    self.model.to(device)
    self.model.eval()
    
    # Create DataLoader for the dataset
    dataloader = DataLoader(self.train_ds, batch_size=batch_size, shuffle=False)
    
    all_probs = []  # To store probabilities or log-probabilities for all batches
    
    with torch.no_grad():  # Disable gradient computation
        for batch in dataloader:  # Assuming dataset returns (inputs, labels)
            inputs = batch['x'].to(device)  # Move to model's device
            
            logits = self.model(inputs)
            
            if return_log_probs:
                # Convert logits to log-probabilities
                batch_log_probs = F.log_softmax(logits, dim=-1)
                all_probs.append(batch_log_probs)
            else:
                # Convert logits to probabilities
                batch_probs = F.softmax(logits, dim=-1)
                all_probs.append(batch_probs)
    
    self.model.to('cpu')
    # Concatenate all batch probabilities/log-probabilities
    return torch.cat(all_probs, dim=0)


In [ ]:
def compute_grad(self: PadgAgent,
                 ):
    # compute the graident f R(w, A) w.r.t w
    pass

In [ ]:
#| export
@patch
def aggregate(self: PadgAgent, lst_active_ids, comm_round, len_clients_ds, one_model= False):
    
    visited = []
    for i, id in enumerate(lst_active_ids):

        neighbour_ids = torch.where(self.connections[id] != float(0))[0]

        model_path = os.path.join(self.cfg.save_dir, 
                                   str(comm_round),
                                   f"local_output_{id}",
                                   "pytorch_model.pth")
        client_state_dict = torch.load(model_path, map_location='cpu', weights_only= False)
        self.model.load_state_dict(client_state_dict)
        
        neighbours_sum = {
            key: torch.zeros_like(value) 
            for key, value in client_state_dict.items()
        }
            
        probs_1 = self.compute_probs(batch_size=32, return_log_probs=True)
        
        for other_id in neighbour_ids:

            if (other_id, id) in visited:
                continue
            other_model_path = os.path.join(self.cfg.save_dir, 
                                    str(comm_round),
                                    f"local_output_{other_id}",
                                    "pytorch_model.pth")
            
            other_client_state_dict = torch.load(other_model_path, map_location='cpu', weights_only= False)
            self.model.load_state_dict(other_client_state_dict)
            
            probs_2 = self.compute_probs(batch_size=32, return_log_probs=False)

            kl_div = F.kl_div(probs_1, probs_2, reduction= 'batchmean').to('cpu')
            self.connections[id][other_id] -= self.cfg.server_lr * self.cfg.lambda_ * kl_div

            # apply constraints to the KL divergence
            self.connections[id][other_id] = self.apply_constraints(self.connections[id][other_id])
            self.connections[id][other_id] = self.connections[other_id][id]

            visited.append((id, other_id))
            visited.append((other_id, id))
            
        for other_id in neighbour_ids:
            other_model_path = os.path.join(self.cfg.save_dir, 
                                    str(comm_round),
                                    f"local_output_{other_id}",
                                    "pytorch_model.pth")
            other_client_state_dict = torch.load(other_model_path, map_location='cpu', weights_only= False)

            weight = self.connections[id][other_id]
            for key in other_client_state_dict.keys():
                neighbours_sum[key].data += weight * other_client_state_dict[key].data

        # for key in neighbours_sum.keys():
            # neighbours_sum[key].data /= len(neighbour_ids)

        for key in client_state_dict.keys():
            client_state_dict[key].data = self.cfg.beta * client_state_dict[key].data + (1 - self.cfg.beta) * neighbours_sum[key].data

    
        # save the updated model to the disk
        self.save_state(client_state_dict, comm_round + 1, id)
        

## MIRA Agent

Mira clients have more parameters. Since it's a client for LLM in principle, we need to feed the generation dataset (the dataset of text ids at the end layer not the logits). Also, a tokenizer and a collate function that will be used for the generation and the data loader construction processes.

In [ ]:
#| export
class AgentMira(FLAgent):
    def __init__(self,
                 data_dict: dict,
                 model: torch.nn.Module,
                 criterion,
                 optimizer: torch.optim.Optimizer,
                 id: int,
                 gen_data_dict: dict,
                 tokenizer: AutoTokenizer,
                 collat_fn: LLMDataCollator,
                 cfg: DictConfig) -> None:
            
        super().__init__(data_dict, model, criterion, optimizer, id)
        
        self.train_ds_genr = gen_data_dict['train']
        self.test_ds_genr = gen_data_dict['test']
        self.tokenizer = tokenizer
        self.collat_fn = collat_fn
        self.cfg = cfg 

In order for us to save space, we will replace the original model with only the trainable peft model parameters. 

### Testing Mira Client

We will do the following:
- Define a Mira client.
- inspect the `init_local_train` and `terminate_local_train` methods and their effect on the model's parameters.

In [ ]:
# #| hide
# from transformers import AutoModelForCausalLM
# gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")
# base_model = deepcopy(gpt2)

In [ ]:
# #| hide
# config = LoraConfig(
#     r=8,# arbitrary numbr but usually 8, 16, 32, 64, 128
#     target_modules=['c_attn'],
#     lora_alpha=8,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     )

# peft_model = get_peft_model(gpt2, config)
# mira  = AgentMira(DataDict, peft_model, criterion, optimizer, 0, train_dataset, test_dataset, None, None, None, None)

/home/ahmed/miniconda3/envs/fedai/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Let us inpect the model architecture:

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()